In [ ]:
```xml
<!-- filepath: /Users/markheadd/Repos/usai-api/examples/comprehensive-example.ipynb -->
<VSCode.Cell language="markdown">
# USAi API Node.js Module - Comprehensive Example

This notebook demonstrates advanced features of the USAi API using our Node.js module, based on the patterns from the official Python documentation.

## Features Covered:
- Authentication and basic setup
- Chat completions with system prompts
- Image analysis for government documents
- Document processing (PDFs)
- Embeddings for semantic search and clustering
- Model differences and capabilities
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Import the USAi API module and required dependencies
import { USAiAPI } from '../src/index.js';
import fs from 'fs';
import path from 'path';

// Setup API client
const client = new USAiAPI({
  apiKey: process.env.USAI_API_KEY || 'your-api-key',
  baseUrl: process.env.USAI_BASE_URL || 'https://your-agency.usai.gov'
});

console.log('✅ USAi API client initialized');
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. Basic Authentication Test

First, let's verify our authentication is working by listing available models.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Test authentication and list models
try {
  const models = await client.getModels();
  console.log('🤖 Available Models:');
  models.data.forEach(model => {
    console.log(`  - ${model.id} (${model.owned_by})`);
  });
} catch (error) {
  console.error('❌ Authentication failed:', error.message);
}
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 2. Minimal Chat Example

The simplest possible chat with just a model and a user prompt, following the Python example pattern.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Minimal chat example - equivalent to Python example
const minimalResponse = await client.createChatCompletion({
  model: 'claude-3-haiku',  // Note: Using underscore format like Python example
  messages: [
    {
      role: 'user',
      content: 'Hello'
    }
  ]
});

console.log('💬 Minimal Chat Response:');
console.log(minimalResponse.choices[0].message.content);
console.log('\n📊 Usage:', minimalResponse.usage);
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 3. Using System Prompts

The messages array can contain multiple content items. Including a system prompt influences how the model behaves - here we'll use a government-appropriate example.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Government-appropriate system prompt example
const systemPromptResponse = await client.createChatCompletion({
  model: 'claude-3-5-sonnet',
  messages: [
    {
      role: 'system',
      content: 'You are a helpful federal government assistant specializing in public policy and civic information. Provide accurate, authoritative responses.'
    },
    {
      role: 'user',
      content: 'Explain the process for filing a Freedom of Information Act (FOIA) request.'
    }
  ]
});

console.log('🏛️ Government Assistant Response:');
console.log(systemPromptResponse.choices[0].message.content);
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 4. Image Analysis Example

Images must be encoded as data URIs. This example shows how to analyze government documents or forms using image inputs.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Helper function to encode image as base64 data URI
function encodeImageAsDataUri(imagePath, mimeType = 'image/jpeg') {
  try {
    const imageBuffer = fs.readFileSync(imagePath);
    const base64Data = imageBuffer.toString('base64');
    return `data:${mimeType};base64,${base64Data}`;
  } catch (error) {
    console.log('Note: Image file not found, using placeholder');
    return 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/wAARCAABAAEDASIAAhEBAxEB/8QAFQABAQAAAAAAAAAAAAAAAAAAAAv/xAAhEAACAQMDBQAAAAAAAAAAAAABAgMABAUGIWEREiMxUf/EABUBAQEAAAAAAAAAAAAAAAAAAAMF/8QAGhEAAgIDAAAAAAAAAAAAAAAAAAECEgMRkf/aAAwDAQACEQMRAD8A';
  }
}

// Analyze government document or form
const imageAnalysisResponse = await client.createChatCompletion({
  model: 'gemini-2.0-flash',
  messages: [
    {
      role: 'system',
      content: 'You are an expert at analyzing government documents and forms. Provide detailed, accurate descriptions.'
    },
    {
      role: 'user',
      content: [
        {
          type: 'text',
          text: 'What type of government document or form is this? Please identify any key elements, seals, or official markings.'
        },
        {
          type: 'image_url',
          image_url: {
            url: encodeImageAsDataUri('path/to/government-document.jpg'),
            detail: 'high'
          }
        }
      ]
    }
  ],
  max_tokens: 300
});

console.log('🖼️ Document Analysis:');
const imageContent = imageAnalysisResponse.choices[0].message.content;
console.log(typeof imageContent === 'string' ? imageContent : 'Analysis completed');
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 5. Document Processing (PDF Analysis)

This demonstrates processing PDF documents, useful for analyzing reports, forms, or policy documents.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Helper function to encode PDF as base64 data URI
function encodePdfAsDataUri(pdfPath) {
  try {
    const pdfBuffer = fs.readFileSync(pdfPath);
    const base64Data = pdfBuffer.toString('base64');
    return `data:application/pdf;base64,${base64Data}`;
  } catch (error) {
    console.log('Note: PDF file not found, using placeholder');
    return 'data:application/pdf;base64,JVBERi0xLjQKJYGBgYEKCjEgMCBvYmoKPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDIgMCBSCj4+CmVuZG9iago=';
  }
}

// Process government PDF document
const documentResponse = await client.createChatCompletion({
  model: 'gemini-2.0-flash', // Gemini has better document processing capabilities
  messages: [
    {
      role: 'user',
      content: [
        {
          type: 'text',
          text: 'Please analyze this government document and provide a comprehensive summary including key policies, dates, and action items.'
        },
        {
          type: 'file',
          file_name: 'Government_Report.pdf',
          file: {
            file_data: encodePdfAsDataUri('path/to/sample-document.pdf')
          }
        }
      ]
    }
  ]
});

console.log('📄 Document Summary:');
const docContent = documentResponse.choices[0].message.content;
console.log(typeof docContent === 'string' ? docContent : 'Document processed');
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 6. Model Capabilities Comparison

Different models have different strengths. Let's test the same task with different models to understand their capabilities.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Test different models with the same prompt
const testPrompt = "Summarize the key provisions of the Federal Acquisition Regulation (FAR) in 3 bullet points.";

const models = ['claude-3-haiku', 'claude-3-5-sonnet', 'llama-3-2', 'gemini-2.0-flash'];

console.log('🔬 Model Comparison Results:\n');

for (const model of models) {
  try {
    const response = await client.createChatCompletion({
      model: model,
      messages: [
        {
          role: 'system',
          content: 'You are a federal procurement expert. Provide accurate, concise information.'
        },
        {
          role: 'user',
          content: testPrompt
        }
      ],
      max_tokens: 200
    });
    
    console.log(`\n🤖 ${model}:`);
    const content = response.choices[0].message.content;
    console.log(typeof content === 'string' ? content : 'Response received');
    console.log(`Tokens used: ${response.usage.total_tokens}`);
    
  } catch (error) {
    console.log(`❌ ${model}: ${error.message}`);
  }
}
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 7. Embeddings for Government Document Clustering

Create embeddings for government documents and demonstrate semantic clustering, useful for organizing large document repositories.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Sample government documents for embedding
const governmentTexts = [
  "The Federal Acquisition Regulation (FAR) is the primary regulation for use by all federal executive agencies in their acquisition of supplies and services with appropriated funds.",
  "A FOIA request is a request for records made under the Freedom of Information Act. Federal agencies are required to disclose any information requested under FOIA unless it falls under one of nine exemptions.",
  "The General Services Administration (GSA) manages and supports the basic functioning of federal agencies through supplies, communications, and transportation services.",
  "Small Business Administration (SBA) provides support to entrepreneurs and small businesses through loans, grants, and educational programs.",
  "The Office of Management and Budget (OMB) assists the President in oversight of federal departments and agencies.",
  "The Congressional Budget Office (CBO) provides budget and economic information to Congress and assists with federal budget process.",
  "The Securities and Exchange Commission (SEC) protects investors and maintains fair and efficient markets.",
  "The Environmental Protection Agency (EPA) protects human health and the environment by writing and enforcing regulations.",
  "The Department of Homeland Security (DHS) secures the nation from threats and manages immigration and customs enforcement."
];

// Create embeddings for clustering
const embeddingResponse = await client.createEmbedding({
  model: 'cohere-english-v3',
  input: governmentTexts,
  input_type: 'clustering' // Following Python example pattern
});

console.log('🔍 Embeddings Analysis:');
console.log(`✅ Created ${embeddingResponse.data.length} embeddings`);
console.log(`📊 Embedding dimensions: ${embeddingResponse.data[0].embedding.length}`);
console.log(`🎯 Token usage: ${embeddingResponse.usage.total_tokens} tokens`);
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 8. Semantic Search Example

Use embeddings to find the most relevant government documents for a user query.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Helper function for cosine similarity
function cosineSimilarity(vecA, vecB) {
  const dotProduct = vecA.reduce((sum, a, i) => sum + a * vecB[i], 0);
  const magnitudeA = Math.sqrt(vecA.reduce((sum, val) => sum + val * val, 0));
  const magnitudeB = Math.sqrt(vecB.reduce((sum, val) => sum + val * val, 0));
  return dotProduct / (magnitudeA * magnitudeB);
}

// User query for semantic search
const userQuery = "How do I request government records and information?";

// Create embedding for the query
const queryEmbedding = await client.createEmbedding({
  model: 'cohere-english-v3',
  input: userQuery,
  input_type: 'search_query'
});

// Calculate similarities
const similarities = embeddingResponse.data.map((docEmbedding, index) => {
  const similarity = cosineSimilarity(
    queryEmbedding.data[0].embedding,
    docEmbedding.embedding
  );
  return {
    index,
    document: governmentTexts[index],
    similarity: similarity
  };
});

// Sort by similarity and show top results
similarities.sort((a, b) => b.similarity - a.similarity);

console.log(`\n❓ Query: "${userQuery}"`);
console.log('\n🎯 Most relevant documents:');
similarities.slice(0, 3).forEach((result, rank) => {
  console.log(`\n${rank + 1}. Similarity: ${result.similarity.toFixed(4)}`);
  console.log(`   Document: ${result.document.substring(0, 100)}...`);
});
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 9. Streaming Response Example

Demonstrate real-time streaming for long-form content generation, useful for generating reports or documentation.
</VSCode.Cell>

<VSCode.Cell language="javascript">
// Streaming example for generating a government report
console.log('🌊 Streaming Response Example:');
console.log('Generating a federal agency overview...\n');

const stream = await client.createChatCompletionStream({
  model: 'claude-3-5-sonnet',
  messages: [
    {
      role: 'system',
      content: 'You are a government policy expert. Write comprehensive, accurate overviews of federal programs.'
    },
    {
      role: 'user',
      content: 'Write a comprehensive overview of the Small Business Administration, including its mission, key programs, and how it supports entrepreneurs.'
    }
  ],
  max_tokens: 500,
  temperature: 0.3
});

let streamedContent = '';
for await (const chunk of stream) {
  const content = chunk.choices[0]?.delta?.content;
  if (content) {
    process.stdout.write(content);
    streamedContent += content;
  }
}

console.log('\n\n✅ Streaming complete!');
console.log(`📝 Total characters streamed: ${streamedContent.length}`);
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 10. Error Handling and Rate Limiting

Demonstrate proper error handling and rate limiting strategies for production government applications.
</VSCode.Cell>

<VSCode.Cell language="javascript">
import { 
  USAiRateLimitError, 
  USAiAuthenticationError,
  USAiAPIError 
} from '../src/index.js';

// Example of comprehensive error handling
async function robustApiCall(prompt, retries = 3) {
  for (let attempt = 1; attempt <= retries; attempt++) {
    try {
      const response = await client.complete(
        'claude-3-haiku',
        prompt,
        { maxTokens: 100 }
      );
      return response;
      
    } catch (error) {
      console.log(`\n🔄 Attempt ${attempt} failed:`);
      
      if (error instanceof USAiRateLimitError) {
        console.log(`⏳ Rate limited. Retry after: ${error.retryAfter || 60} seconds`);
        if (attempt < retries) {
          await new Promise(resolve => setTimeout(resolve, (error.retryAfter || 60) * 1000));
          continue;
        }
      } else if (error instanceof USAiAuthenticationError) {
        console.log('🔑 Authentication failed. Check API key.');
        break; // Don't retry auth errors
      } else if (error instanceof USAiAPIError) {
        console.log(`⚠️ API Error: ${error.message} (${error.type})`);
      } else {
        console.log(`❌ Unexpected error: ${error.message}`);
      }
      
      if (attempt === retries) {
        throw error;
      }
    }
  }
}

// Test error handling
try {
  const result = await robustApiCall("What is the Federal Register?");
  console.log('\n✅ Robust API call succeeded:');
  console.log(result);
} catch (error) {
  console.log('\n❌ All retry attempts failed:', error.message);
}
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Summary

This comprehensive example demonstrates:

1. ✅ **Authentication** - Proper API key setup and validation
2. ✅ **Basic Chat** - Simple conversations with AI models
3. ✅ **System Prompts** - Configuring AI behavior for government use
4. ✅ **Image Analysis** - Processing government documents and forms
5. ✅ **Document Processing** - Analyzing PDFs and files
6. ✅ **Model Comparison** - Understanding different AI capabilities
7. ✅ **Embeddings** - Semantic search and document clustering
8. ✅ **Streaming** - Real-time response generation
9. ✅ **Error Handling** - Robust production-ready patterns

The Node.js module successfully replicates and enhances the patterns shown in the Python documentation while providing additional government-specific features and TypeScript safety.
</VSCode.Cell>
```